In [1]:
import pandas as pd

In [2]:
input_file = "../datasets/final_queries_v1.5.json"
output_file = "../datasets/final_queries_v1.6.json"

df = pd.read_json(input_file)
print(len(df))
df.head()

2842


,query,description,context,prompt,num_tokens,start_with_SELECT
0,SELECT ?property ?propertyType ?propertyLabel ...,Wikidata properties in numerical order,Counting stuff on Wikidata\nAll Wikidata prope...,<s>[INST] <<SYS>>This is a conversation betwee...,267,True
1,SELECT ?id ?idLabel ?idDescription ?new{\n?id ...,Wikidata properties excluding external IDs,Counting stuff on Wikidata\nVariation of the a...,<s>[INST] <<SYS>>This is a conversation betwee...,296,True
2,SELECT (COUNT(?article) AS ?count)\nWHERE {\n?...,,Counting stuff on Wikidata\nCount of scientifi...,<s>[INST] <<SYS>>This is a conversation betwee...,197,True
3,SELECT (COUNT(DISTINCT ?article) AS ?count)\nW...,Count of fictional characters,Counting stuff on Wikidata\nCount of fictional...,<s>[INST] <<SYS>>This is a conversation betwee...,203,True
4,SELECT (COUNT(?item) AS ?count)\nWHERE { ?item...,Count of items with coordinate locations,Counting stuff on Wikidata\nCount of items wit...,<s>[INST] <<SYS>>This is a conversation betwee...,186,True


In [3]:
index_to_drop = [
    1824,
    2097,
    1632,
    2049
]

for i, s in df.loc[index_to_drop].iterrows():
    print(s.query)
    print("------")

SELECT ?item ?itemLabel ?adjacentItem ?adjacentItemLabel
WITH
{
SELECT ?item ?adjacent
WHERE
{
?item wdt:P31 wd:Q27971968.
?item wdt:P3120 ?TOID.
BIND(SUBSTR(?TOID,12,5) AS ?unitID)
SERVICE <http://data.ordnancesurvey.co.uk/datasets/os-linked-data/apis/sparql>
{
[] ag:hasUnitID ?unitID; sr:touches ?adjacent.
}
}
} AS %federated_subquery
WHERE
{
INCLUDE %federated_subquery
BIND(SUBSTR(STR(?adjacent),37,16) AS ?adjacentTOID)
?adjacentItem wdt:P3120 ?adjacentTOID.
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }
}
------
select distinct ?item ?itemLabel (sample(?logo1) as ?logo) (sample(?img) as ?image) (sample(?pos) as ?coord) ?participants
where {
# institutions' items and count of participants from custom WDQS-enabled endpoint
service <http://zbw.eu/beta/sparql/econ_corp/query> {
?item zbwext:swib20participants ?count .
bind(concat(str(?count), ' participant', if(?count > 1, 's', '')) as ?participants)
}
optional { ?item wdt:P18 ?img . }
optional { ?ite

In [4]:
df = df.drop(index_to_drop).reset_index(drop=True)
print(len(df))
df.head()

2838


,query,description,context,prompt,num_tokens,start_with_SELECT
0,SELECT ?property ?propertyType ?propertyLabel ...,Wikidata properties in numerical order,Counting stuff on Wikidata\nAll Wikidata prope...,<s>[INST] <<SYS>>This is a conversation betwee...,267,True
1,SELECT ?id ?idLabel ?idDescription ?new{\n?id ...,Wikidata properties excluding external IDs,Counting stuff on Wikidata\nVariation of the a...,<s>[INST] <<SYS>>This is a conversation betwee...,296,True
2,SELECT (COUNT(?article) AS ?count)\nWHERE {\n?...,,Counting stuff on Wikidata\nCount of scientifi...,<s>[INST] <<SYS>>This is a conversation betwee...,197,True
3,SELECT (COUNT(DISTINCT ?article) AS ?count)\nW...,Count of fictional characters,Counting stuff on Wikidata\nCount of fictional...,<s>[INST] <<SYS>>This is a conversation betwee...,203,True
4,SELECT (COUNT(?item) AS ?count)\nWHERE { ?item...,Count of items with coordinate locations,Counting stuff on Wikidata\nCount of items wit...,<s>[INST] <<SYS>>This is a conversation betwee...,186,True


In [5]:
df.to_json(output_file)